In [2]:
# importing all necessary libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import treebank
from nltk.grammar import PCFG, induce_pcfg, toy_pcfg1, toy_pcfg2
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# from nltk.stem.porter import PorterStemmer
# import re
# from bs4 import BeautifulSoup #To remove HTML tags

Step#1: Read and store all positive reviews in a list called ‘pos’ and all negative reviews in a list called ‘neg’. 
Create a Dataframe with all reviews under column ‘Review’, add a new column ‘class’ assigning label '1' for 
positive reviews and '0' for negative reviews (4 Marks)

In [3]:
# To read and store all positive reviews in pos and neg separately
df = pd.read_csv('D://M.Tech//Sem 3//Natural Language processing//imdb-dataset-of-50k-movie-reviews//IMDB Dataset.csv')
# df.head()
pos=df["review"][df["sentiment"]=='positive']
pos.tolist()
neg=df["review"][df["sentiment"]=='negative']
neg.tolist()
lbl = {"positive": 1, "negative": 0}
dataset=pd.DataFrame(df).rename(columns={'review': 'Review', 'sentiment': 'class'}).replace({"class": lbl}) 
dataset.head(20)


,Review,class
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1
6,I sure would like to see a resurrection of a u...,1
7,"This show was an amazing, fresh & innovative i...",0
8,Encouraged by the positive comments about this...,0
9,If you like original gut wrenching laughter yo...,1


In [4]:
dataset.describe()

,class
count,50000.000000
mean,0.500000
std,0.500005
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


Step#2:Remove punctuations and stopwords from the text in ‘Review’ column (4 Marks)

In [5]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')

stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub("<.*?>", "",text)
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

dataset['Processed_Reviews'] = dataset.Review.apply(lambda x: clean_text(x))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\subba\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
dataset.head()

,Review,class,Processed_Reviews
0,One of the other reviewers has mentioned that ...,1,one reviewer ha mention watch 1 oz episode you...
1,A wonderful little production. <br /><br />The...,1,wonderful little production film technique una...
2,I thought this was a wonderful way to spend ti...,1,think wa wonderful way spend time hot summer w...
3,Basically there's a family where a little boy ...,0,basically family little boy jake think zombie ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,petter matteis love time money visually stun f...


Step#3:Create two objects X and y. X will be the ' Review' column of the above dataframe and y will be the 'class' column.
Create a CountVectorizer object and split the data into training and testing sets. Train a MultinomialNB model for 
classifying the reviews and Display the confusion Matrix (5 Marks)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
# Creating the Bag of Words model
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(dataset['Processed_Reviews']).toarray()
dataset
y = dataset.iloc[0:50000, 1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

In [8]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [9]:
y_pred=clf.predict(X_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[4203,  832],
       [ 838, 4127]], dtype=int64)

In [61]:
accuracy=(4203+4127)/(10000)

In [62]:
print(accuracy)

0.833


Step#4:Display the HMM POS tagging on the first 4 rows of ‘Review’ (3 Marks)

In [10]:
pd.set_option('max_colwidth', 2000)
dataset["Processed_Reviews"].iloc[:4]

0    one reviewer ha mention watch 1 oz episode youll hook right exactly happen methe first thing strike oz wa brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use wordit call oz nickname give oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home manyaryans muslim gangsta latino christian italian irish moreso scuffle death stare dodgy deal shady agreement never far awayi would say main appeal show due fact go show wouldnt dare. forget pretty picture paint mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw strike nasty wa surreal, say wa ready it, watch more, develop taste oz, get accustom high level graphic violence. violence, injustice (crooked guard who'll sell nickel, inmate who'll kill order get away it, well mannered, middle class inmate turn p

In [11]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
fourrows=dataset["Processed_Reviews"].iloc[:4].to_string(index=False)
# print(fourrows)
tokens=nltk.word_tokenize(fourrows)

print("Parts of speech for first four lines is: \n \n",  nltk.pos_tag(tokens))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\subba\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\subba\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Parts of speech for first four lines is: 
 
 [('one', 'CD'), ('reviewer', 'JJR'), ('ha', 'NN'), ('mention', 'NN'), ('watch', 'NN'), ('1', 'CD'), ('oz', 'JJ'), ('episode', 'NN'), ('youll', 'NN'), ('hook', 'NN'), ('right', 'RB'), ('exactly', 'RB'), ('happen', 'VB'), ('methe', 'NN'), ('first', 'JJ'), ('thing', 'NN'), ('strike', 'NN'), ('oz', 'VBP'), ('wa', 'JJ'), ('brutality', 'NN'), ('unflinching', 'VBG'), ('scene', 'NN'), ('violence', 'NN'), ('set', 'VBN'), ('right', 'RB'), ('word', 'NN'), ('go', 'VB'), ('trust', 'NN'), ('show', 'NN'), ('faint', 'NN'), ('hearted', 'VBD'), ('timid', 'JJ'), ('show', 'NN'), ('pull', 'JJ'), ('punch', 'JJ'), ('regard', 'JJ'), ('drug', 'NN'), ('sex', 'NN'), ('violence', 'NN'), ('hardcore', 'NN'), ('classic', 'JJ'), ('use', 'NN'), ('wordit', 'NN'), ('call', 'NN'), ('oz', 'NN'), ('nickname', 'NN'), ('give', 'JJ'), ('oswald', 'NN'), ('maximum', 'JJ'), ('security', 'NN'), ('state', 'NN'), ('penitentary', 'JJ'), ('focus', 'NN'), ('mainly', 'RB'), ('emerald', 'VBZ'

Step#5:Parse the first 4 rows of ‘Review’ using Viterbi Parser 
[Use toy_pcfg1 and toy_pcfg2 to get the probabilistic context free grammars; use the PCFG suitable for each sentence] (4 marks)

In [16]:
grammar=toy_pcfg2

In [17]:
print(grammar)

Grammar with 23 productions (start state = S)
    S -> NP VP [1.0]
    VP -> V NP [0.59]
    VP -> V [0.4]
    VP -> VP PP [0.01]
    NP -> Det N [0.41]
    NP -> Name [0.28]
    NP -> NP PP [0.31]
    PP -> P NP [1.0]
    V -> 'saw' [0.21]
    V -> 'ate' [0.51]
    V -> 'ran' [0.28]
    N -> 'boy' [0.11]
    N -> 'cookie' [0.12]
    N -> 'table' [0.13]
    N -> 'telescope' [0.14]
    N -> 'hill' [0.5]
    Name -> 'Jack' [0.52]
    Name -> 'Bob' [0.48]
    P -> 'with' [0.61]
    P -> 'under' [0.39]
    Det -> 'the' [0.41]
    Det -> 'a' [0.31]
    Det -> 'my' [0.28]


In [ ]:
# pcfg1 ="I saw john with telescope"
# "John saw my man under telescope"
# "The man saw john with telescope"

In [ ]:
# pcfg2="the boy saw a cookie under my table"
# "the boy saw hill with telescope"
# "the boy saw Jack with Bob under the table with a telescope"

In [58]:
# demo = [('I saw John with my telescope', toy_pcfg1)]
# sent, grammar = demos[0]
# As we are getting errors when we parse Review column so we couldnt input it.
# As per the direction by the instructor taking 3 sample statements each for toy_pcfg1 and toy_pcfg2 
# (same was conveyed to us in Canvas)
from nltk.parse import ViterbiParser
demos = [('I saw the man with my telescope', toy_pcfg1),("John saw my man under telescope", toy_pcfg1),
(" man saw with my telescope",toy_pcfg1),("the boy saw Jack with Bob under the table with a telescope", toy_pcfg2),        
        ("the boy saw a cookie under my table",toy_pcfg2),("the boy saw hill with telescope",toy_pcfg2)] 

for i in range(len(demos)): 
    sent=demos[i][0]
    grammar=demos[i][1]
    tokens = sent.split() 
    parser = ViterbiParser(grammar) 
    all_parses = {} 
    print('\nsentence: %s\nparser: %s\ngrammar: %s' % (sent,parser,grammar))
    parser.trace(3)
    t = time.time()
    parses = parser.parse_all(tokens)
    times.append(time.time()-t)
    if parses: 
        lp = len(parses)
        p = reduce(lambda a,b:a+b.prob(), parses, 0.0)
    else: 
        p = 0
    average_p.append(p)
    num_parses.append(len(parses))
    for p in parses: 
        all_parses[p.freeze()] = 1


sentence: I saw the man with my telescope
parser: <ViterbiParser for <Grammar with 17 productions>>
grammar: Grammar with 17 productions (start state = S)
    S -> NP VP [1.0]
    NP -> Det N [0.5]
    NP -> NP PP [0.25]
    NP -> 'John' [0.1]
    NP -> 'I' [0.15]
    Det -> 'the' [0.8]
    Det -> 'my' [0.2]
    N -> 'man' [0.5]
    N -> 'telescope' [0.5]
    VP -> VP PP [0.1]
    VP -> V NP [0.7]
    VP -> V [0.2]
    V -> 'ate' [0.35]
    V -> 'saw' [0.65]
    PP -> P NP [1.0]
    P -> 'with' [0.61]
    P -> 'under' [0.39]
Inserting tokens into the most likely constituents table...
   Insert: |=......| I
   Insert: |.=.....| saw
   Insert: |..=....| the
   Insert: |...=...| man
   Insert: |....=..| with
   Insert: |.....=.| my
   Insert: |......=| telescope
Finding the most likely constituents spanning 1 text elements...
   Insert: |=......| NP -> 'I' [0.15]               0.1500000000 
   Insert: |.=.....| V -> 'saw' [0.65]              0.6500000000 
   Insert: |.=.....| VP -> V [0.